In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
from pathlib import Path

In [ ]:
from src.features.data_encoding import numpy_structure

# Constants

In [ ]:
chain_dir = Path("../data/data_preparation/00a-pdbs_chains/")
pid = "6BOY"
cid = "C"

# Read

In [ ]:
p_filename = chain_dir / f"{pid}_{cid}.pdb"
protein_encoded = numpy_structure(str(p_filename))
xyz = protein_encoded[-1]  # coordinates

# Compute points and normals

In [ ]:
# P[cid], n, bm = atoms_to_points_normals(xyz_pt, b)

In [ ]:
atoms = tf.convert_to_tensor(xyz)
# batch,
distance=1.05
smoothness=0.5
resolution=1.0
nits=4
atomtypes=None
sup_sampling=20
variance=0.1

In [ ]:
T = distance

N, D = atoms.shape
B = sup_sampling  # Sup-sampling ratio

In [ ]:
g1 = tf.random.Generator.from_seed(1, alg='philox')
random_dists = 0.2 * T * g1.normal(shape=[N, B, D])
z = atoms[:,None,:] + random_dists

In [ ]:
z = tf.reshape(z, [-1, D])

In [ ]:
z

In [ ]:
mask = tf.math.logical_and(z[:,0] > 65, z[:,0] < 65.5)

In [ ]:
mask

In [ ]:

plt.plot( tf.boolean_mask(z[:,1], mask), tf.boolean_mask(z[:,2], mask), '.')
# plt.plot( xyz[N_atom,0], xyz[N_atom,1], 'o')

In [ ]:
atoms.shape

In [ ]:
def calcf(x):
    y = - 1.5 *x* x + 0.2 *x + 2
    return y

In [ ]:
x = np.linspace(-5,5, num=30)
y = calcf(x)

In [ ]:
plt.plot(x,y, '--')
plt.show()

In [ ]:
a = tf.random.normal([20,2]) * np.array([1.5, 4]) - np.array([1, 8])

In [ ]:
plt.plot(x,y, '--')
plt.plot(a[:,0], a[:,1], '.')
plt.plot(a[:,0], calcf(a[:,0]), 'o')
plt.show()

In [ ]:
b = tf.Variable(a[:,0])

In [ ]:
for i in range(50):
    with tf.GradientTape() as tape:
        tape.watch(b)
        rb = calcf(b)
        errors = rb - a[:,1]
        # This is the same mean-squared-error calculation as before
        loss = tf.reduce_mean(errors**2)

    # Auto-diff magic!  Calcs gradients between loss calc and params
    dloss_dparams = tape.gradient(loss, [b])

    # # Gradients point towards +loss, so subtract to "descend"
    b = b - 0.01 * dloss_dparams[0]
# tf_icept = tf_icept - learning_rate * dloss_dparams[1]

In [ ]:
plt.plot(x,y, '--')
plt.plot(b, a[:,1], '.')
# plt.plot(a[:,0], calcf(a[:,0]), 'o')
plt.show()

In [ ]:
c